## Get Adidas tweets with replies using twitter API Tweepy

- Fetch the tweets from **@adidas** using tweepy api user_timeline
- Find the `tweet_id=tweet.id` of the tweets for which the replies are need to be fetched
- Using tweepi api search retrieve all the tweets matching (`q='to:@adidas'`) tweets
- Results matching the `in_reply_to_status_id_str` to `tweet_id` is the replies for the post


In [1]:
# import required packages
import tweepy
from tweepy import OAuthHandler
import pandas as pd
import numpy as np

In [2]:
# Twitter API Credentials
access_token = '3147900582-iyD6Mlvi2YjdXrM5OQZxAAgZ0y2uO3NlLJkGKVG'
access_secret = 'LfEKItqGYoUmBLRAP29ELviAwE8lqiO1WBfny9FtoRip8'
consumer_key = 'QvAIXD2iYvJeWyIhcMyA7Inhf'
consumer_secret = '0VJkkwkatSmfvez0k8M40G3XPktXCrVz4euxylAB1WCLCisKWX'

#create connection with Twitter API
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [3]:
# Function to retrieve replies to specific tweet
def get_replies_q(tweet_id, reply_count, reply_texts):
    #get the most recent tweets posted for the keyword specified
    for tweet in tweepy.Cursor(api.search, q='to:@adidas', result_type='recent', tweet_mode='extended', timeout=999999).items(1000):
        if hasattr(tweet, 'in_reply_to_status_id_str'):
            # Compare the given tweet id with queries reply to tweet id
            if(tweet.in_reply_to_status_id_str==tweet_id):
                reply_count+=1
                reply_texts.append(tweet.full_text)
    return reply_count

In [4]:
username='adidas'
replies = []
tweets_list=[]

try:
    # get the most recent 5 tweets posted from the user specified
    tw_obj1 = tweepy.Cursor(api.user_timeline,id=username, tweet_mode='extended',timeout=999999).items(5)
    for tweet in tw_obj1:
        reply_texts, reply_count=[],0
        tweet_url = f"https://twitter.com/adidas/status/{tweet.id}"      
        # Function call to get replies for the passed tweet id
        rep = get_replies_q(str(tweet.id),reply_count,reply_texts)     
        # Append the retrieved data to list
        tweets_list.append([tweet.created_at, tweet.id, tweet.full_text, tweet.user._json['screen_name'], tweet.user._json['name'],
                            tweet_url, tweet._json['favorite_count'], tweet._json['retweet_count'], rep, reply_texts])
        
    # Create new dataframe for the collected tweet data
    tweets_df = pd.DataFrame(tweets_list,columns = ['created_at','tweet_id', 'tweet_text', 'screen_name', 'name', 'tweet_url',
                                                    'favorite_count', 'retweet_count','reply_count','replies_list'])
    
except BaseException as e:
    print('something went wrong', str(e))

In [5]:
tweets_df.head()

,created_at,tweet_id,tweet_text,screen_name,name,tweet_url,favorite_count,retweet_count,reply_count,replies_list
0,2022-02-25 22:17:47,1497335041214590981,RT @adidasoriginals: adidas x Gucci \n\n#adida...,adidas,adidas,https://twitter.com/adidas/status/149733504121...,0,4736,0,[]
1,2022-02-22 13:38:03,1496117082068500480,A limited number of the new adidas TERREX HS1 ...,adidas,adidas,https://twitter.com/adidas/status/149611708206...,34,4,5,"[@adidas If you have trade in Russia, we ask ..."
2,2022-02-22 13:38:02,1496117077752664065,Introducing: the adidas TERREX HS1\n \nCompose...,adidas,adidas,https://twitter.com/adidas/status/149611707775...,68,8,5,"[@adidas As a tier 1 sponsor of FIFA, what do ..."
3,2022-02-22 13:38:00,1496117070504906752,ICYMI: Last year we partnered with Finnish tex...,adidas,adidas,https://twitter.com/adidas/status/149611707050...,18,1,3,"[@adidas @SpinnovaPlc Looks interesting 🤔, @ad..."
4,2022-02-22 13:38:00,1496117068864831495,"Calling all adventurers, hikers, and nature lo...",adidas,adidas,https://twitter.com/adidas/status/149611706886...,102,11,15,"[@adidas If you have trade in Russia, we ask ..."


In [6]:
# Lets look at the retrieved replies
pd.set_option('display.max_colwidth', None)
tweets_df.replies_list

0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [7]:
# Lets look at the retrieved replies specific tweet
pd.set_option('display.max_colwidth', None)
tweets_df.replies_list[2]

['@adidas As a tier 1 sponsor of FIFA, what do you make of FIFA’s continuing support for Putin’s murderous regime? Will your brand be lining the side of the pitch at Russia’s next game? Image as the Russian players celebrate a goal right next to your brand - make a stand! Cancel your ties!',
 '@adidas You guys should enter the usership-economy to the very fullest extent, by offering Adidas-subscriptions with a wide spectrum of packages.\n\nhttps://t.co/RuN7D1mQK9',
 '@adidas Your prayers are not enough.\n🇺🇦🇺🇦🇺🇦 DONT STARE. HELP UKRAINE. 🇺🇦🇺🇦🇺🇦 #dont_stare_help_ukraine',
 '@adidas 🇺🇦🇺🇦🇺🇦DON’T STARE. HELP UKRAINE 🇺🇦🇺🇦🇺🇦\n\nRUSSIAN FORCES IS TRYING TO SEIZE CHORNOBYL NUCLEAR PLANT IN ORDER TO MAKE ECOLOGICAL DISASTER',
 'A limited number of the new adidas TERREX HS1 will be made commercially available on https://t.co/obaGevq34N and in other selected retail outlets from July.']

In [10]:
# Store the dataset to csv
tweets_df.to_csv(path_or_buf = 'data/adidas.csv', encoding='utf-8', index=False)